In [1]:
import graphlab
graphlab.canvas.set_target('ipynb')

# Put our product reviews in a SFrame

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to znorris@gmail.comand will expire on October 12, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-8520 - Server binary: /home/znorris/anaconda/envs/dato-env/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1445843077.log
[INFO] GraphLab Server Version: 1.6.1


# Define what's positive and a negative sentiment

In [3]:
# ignore all 3 star reviews, those people just can't make up their minds
products = products[products['rating'] != 3]
# positive sentiment = 4 or 5 star reviews
products['sentiment'] = products['rating'] >= 4

In [4]:
# take a peek, make sure that worked
products.head()

name,review,rating,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,1


In [5]:
# Create a new column for total word counts / review
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

# define a subset of 'polorized words' to be used in a second model
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [6]:
# take a peek, did we create our new column with word counts?
products.head()

name,review,rating,sentiment,word_count
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,1,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,1,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,1,"{'ingenious': 1, 'and':3, 'love': 2, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,1,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,1,"{'and': 2, 'cute': 1,'help': 2, 'doll': 1, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,1,"{'shop': 1, 'be': 1,'is': 1, 'it': 1, 'as': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,1,"{'feeding,': 1, 'and': 2,'all': 1, 'right': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,1,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,1,"{'journal.': 1, 'all': 1,'standarad': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,1,"{'all': 1, 'forget': 1,'just': 1, ""daughter's"": ..."


# 1. Build new features for modeling

In [7]:
# Create a function that will look through a dictionary, where the dictionary keys 
# are words and their values are the number of times that word appears. That function will then
# return the value for that word. (useful for filtering words from a 'bag-of-words')
# 
def awesome_count(dict, word):
    if word in dict:
        return dict[word]
    else:
        return 0

In [8]:
# Take our list of words and create a new column for each of them. In that column  
# put an integer that is the number of times that word appears. Needed to use a
# lambda function so that a second input can be specified.
for word in selected_words:
    print('Currently running against: {} of {}').format(word, type(word))  # A little debug action
    products[word] = products['word_count'].apply(lambda dict: awesome_count(dict, word))
#products['all'] = products['word_count'].apply(lambda dict: awesome_count(dict, 'all'))

Currently running against: awesome of <type 'str'>
Currently running against: great of <type 'str'>
Currently running against: fantastic of <type 'str'>
Currently running against: amazing of <type 'str'>
Currently running against: love of <type 'str'>
Currently running against: horrible of <type 'str'>
Currently running against: bad of <type 'str'>
Currently running against: terrible of <type 'str'>
Currently running against: awful of <type 'str'>
Currently running against: wow of <type 'str'>
Currently running against: hate of <type 'str'>


In [9]:
# Let's take a look and see that everything worked.
# Take each column that was created and count up the total number of products
# (or rows) that used this word.
for word in selected_words:
    if products[word]:
        print('Number of products with the word "{}": {}').format(word, len(products[products[word] >= 1]))

Number of products with the word "awesome": 1931
Number of products with the word "great": 35660
Number of products with the word "fantastic": 849
Number of products with the word "amazing": 1256
Number of products with the word "love": 33165
Number of products with the word "horrible": 623
Number of products with the word "bad": 3019
Number of products with the word "terrible": 643
Number of products with the word "awful": 327
Number of products with the word "wow": 119
Number of products with the word "hate": 998


In [10]:
# Double check all of our new columns are there.
products.head()

name,review,rating,sentiment,word_count,awesome
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,1,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ...",0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,1,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,1,"{'ingenious': 1, 'and':3, 'love': 2, ...",0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,1,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ...",0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,1,"{'and': 2, 'cute': 1,'help': 2, 'doll': 1, ...",0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,1,"{'shop': 1, 'be': 1,'is': 1, 'it': 1, 'as': ...",0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,1,"{'feeding,': 1, 'and': 2,'all': 1, 'right': 1, ...",0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,1,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ...",0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,1,"{'journal.': 1, 'all': 1,'standarad': 1, ...",0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,1,"{'all': 1, 'forget': 1,'just': 1, ""daughter's"": ...",0


## Question 1:
Using the .sum() method on each of the new columns you created, answer the following questions: 

Out of the selected_words, which one is most used in the dataset? *"great"*

Which one is least used? *"wow"*

In [11]:
for word in selected_words:
    if products[word]:
        print('Total number of times the word "{}" was used: {}').format(word, products[word].sum())

Total number of times the word "awesome" was used: 2002
Total number of times the word "great" was used: 42420
Total number of times the word "fantastic" was used: 873
Total number of times the word "amazing" was used: 1305
Total number of times the word "love" was used: 40277
Total number of times the word "horrible" was used: 659
Total number of times the word "bad" was used: 3197
Total number of times the word "terrible" was used: 673
Total number of times the word "awful" was used: 345
Total number of times the word "wow" was used: 131
Total number of times the word "hate" was used: 1057


# 2. Create new sentiment analysis models

In [12]:
# Split our data into two subsets
train_data, test_data = products.random_split(.8, seed=0)

# Create a list where each element in the list is a string, that string
# must be the name of a column in our SFrame.
basic_features = ['word_count']
advanced_features = selected_words

# create our simple model based on all word counts
sentiment_model = graphlab.logistic_classifier.create(train_data, 'sentiment', 
                                                      features=basic_features, 
                                                      validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 2.215567     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 3.374844     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 3.829812     | 0.92

In [13]:
# create our advanced model based on our list of key words
selected_words_model = graphlab.logistic_classifier.create(train_data, 'sentiment', 
                                                           features=advanced_features, 
                                                           validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.238556     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 0.356523     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 0.485179     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        |

## Examine the weights assigned by our learned classifier
The ML algorith did this for us.

In [14]:
# learned models have a field 'coefficients' let's take a look
sentiment_model['coefficients']

name,index,class,value
(intercept),None,1,0.729182482603
word_count,it.,1,0.0923459975112
word_count,recommend,1,0.351653944839
word_count,love,1,0.824676597257
word_count,it,1,0.00340245508889
word_count,disappointed.,1,-2.66907012284
word_count,planet,1,-0.28318516271
word_count,and,1,0.0387848304637
word_count,bags,1,0.132287521499
word_count,wipes,1,-0.0146873544927


## Question 2:
Out of the 11 words in selected_words, which one got the most positive weight? *love*

Which one got the most negative weight? *awful*

In [15]:
# learned models have a field 'coefficients' let's take a look
selected_words_model['coefficients'].sort('value', ascending=False)

name,index,class,value
love,None,1,1.39989834302
(intercept),None,1,1.36728315229
awesome,None,1,1.05800888878
amazing,None,1,0.892802422508
fantastic,None,1,0.891303090304
great,None,1,0.883937894898
wow,None,1,-0.0541450123333
bad,None,1,-0.985827369929
hate,None,1,-1.40916406276
awful,None,1,-1.76469955631


# 3. Comparing the accuracy of different sentiment analysis models
### Sentiment Model: All Words

In [16]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.916256305548883, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  1461 |
 |      0       |        1        |  1328 |
 |      0       |        0        |  4000 |
 |      1       |        1        | 26515 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

### Sentiment Model: Selected Words

In [18]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8431419649291376, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  234  |
 |      1       |        0        |  130  |
 |      0       |        1        |  5094 |
 |      1       |        1        | 27846 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

## Question 3:
**Using the evaluations above answer the following questions:**

What is the accuracy of the selected_words_model on the test_data? *0.8431419649291376*

What was the accuracy of the sentiment_model that we learned using all the word counts? *0.916256305548883*

What is the accuracy majority class classifier on this task? *~0.8414*

How do you compare the different learned models with the baseline approach where we are just predicting the majority class? *sentiment_model performed best, next was selected_words_model, however it was only slightly better than the majority class classifier.*

In [37]:
# Determine the accuracy of the majority class classifier
# add the percentages for value 5 and 4, this comprises our positive sentiment.
total_training_reviews = train_data.num_rows()
positive_training_reviews = train_data[train_data['sentiment'] >= 1].num_rows()

In [38]:
print total_training_reviews
print positive_training_reviews
print float(positive_training_reviews) / total_training_reviews

133448
112283
0.841398896949


In [39]:
test_data['sentiment'].show(view='Categorical')

# 4. Interpreting the difference in performance between models

In [19]:
# To compare models we'll create a subset that only includes one product.
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [20]:
# Create a new column for each review of this product type
# in the column we'll put the prediction as probability.
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [21]:
# Let's sort these reviews.
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [22]:
# Take a look to be sure that worked.
diaper_champ_reviews.head()

name,review,rating,sentiment,word_count,awesome
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,1,"{'all': 1, 'less': 1,""friend's"": 1, '(which': ...",0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,1,"{'just': 1, 'over': 1,'rweek': 1, 'sooo': 1, ...",0
Baby Trend Diaper Champ,We researched all of thedifferent types of di ...,4.0,1,"{'all': 2, 'just': 4,""don't"": 2, 'one,': 1, ...",0
Baby Trend Diaper Champ,My baby is now 8 monthsand the can has been ...,5.0,1,"{""don't"": 1, 'when': 1,'over': 1, 'soon': 1, ...",0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,1,"{'just': 3, 'money': 1,'not': 2, 'mechanism' ...",0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,1,"{'all': 1, 'bags.': 1,'son,': 1, '(i': 1, ...",0
Baby Trend Diaper Champ,Wow! This is fabulous.It was a toss-up between ...,5.0,1,"{'and': 4, '""genie"".': 1,'since': 1, 'garbage' ...",0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,1,"{'lysol': 1, 'all': 2,'bags.': 1, 'feedback': ...",0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,1,"{'just': 1, 'when': 1,'both': 1, 'results': 1, ...",0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,1,"{'taller': 1, 'bags.': 1,'just': 1, ""don't"": 4, ...",0


## Question 4:
What's the 'predicted_sentiment' for the most positive review according to the 'sentiment_model' as seen above? *0.999999937267*

In [23]:
# Use our selected words model to get the same info
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.7969408512906712]

## Question 5:
What's the 'predicted_sentiment' for the most positive review according to the 'selected_words_model' as seen above? *0.7969408512906712*

## Question 6:
Why is the predicted_sentiment for the most positive review found using the model with all word counts (sentiment_model) much more positive than the one using only the selected_words (selected_words_model)? *Because there are more words and that will result in a higher total score*

In [40]:
# Let's take a look at that particular review we're getting questions about.
diaper_champ_reviews[0:1]

Columns:
	name	str
	review	str
	rating	float
	sentiment	int
	word_count	dict
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int
	predicted_sentiment	float

Rows: 1

Data:
+-------------------------+-------------------------------+--------+-----------+
|           name          |             review            | rating | sentiment |
+-------------------------+-------------------------------+--------+-----------+
| Baby Trend Diaper Champ | Baby Luke can turn a clean... |  5.0   |     1     |
+-------------------------+-------------------------------+--------+-----------+
+-------------------------------+---------+-------+-----------+---------+------+
|           word_count          | awesome | great | fantastic | amazing | love |
+-------------------------------+---------+-------+-----------+---------+------+
| {'all': 1, 'less': 1, "fri... |    0    |   0   |     0     |    0    |  0   |
+-------------------------------+---------+-------+-----------+---------+------+
+----------+-----+----------+-------+-----+------+---------------------+
| horrible | bad | terrible | awful | wow | hate | predicted_sentiment |
+----------+-----+----------+-------+-----+------+---------------------+
|    0     |  0  |    0     |   0   |  0  |  0   |    0.999999937267   |
+----------+-----+----------+-------+-----+------+---------------------+
[1 rows x 17 columns]